
### Students:

Rui Alves nº 15505

Joel Jonassi nº 19698

### Teacher:

 Joaquim Silva

# CONSTRAINT SATISFACTION PROBLEMS (CSP) - Schedules making problem

# Introduction

This practical work intends to increase the knowledge acquired during Artificial Intelligence classes through the development of an intelligent agent.
In the current scenario of energy crisis, it is intended to reschedule the schedules of classes of the undergraduate courses of the school of technology in order to reduce
travel to campus and to lower the need for air conditioning.

Some constraints:

- All lessons, with a duration of 2 hours, take place on weekdays.

- All classes have 10 lessons per week, of which 1 or 2 lessons are online.

- A class should not have more than 3 lessons per day. Online lessons cannot be booked immediately or immediately after a face-to-face lesson.

- Only up to 2 lessons can take place in the morning and up to 2 lessons in the afternoon.

- Every class has 2 to 4 lessons in a specific classroom.

- Can’t assign a classroom that it is already occupied to another class;

- All lessons must start at least at 9am and end at most by 6pm.

According to the constraints described above we will formulate and implement an agent that finds the best solution for class schedules, minimizing the number of days each
class must travel to campus and the number of classrooms used.
We will end with a small conclusion, where we will summarize the development of the project, alluding to the difficulties encountered.


## Schedules making problem - Problem Formulation

We need to prepare the classes schedules for the next semester, taking into account the following information:

- the list of classes;
- the list of courses that are in the program of each class;
- the available time slots to assign to the classes
- the classrooms to assign to the classes

Variables: timeslot, days, classrooms, curricular units.

The domain of each variable:

- time slot ={ 8, 9, 10, 11, 12, 13, 14, 15, 16, 17 }

- days = { monday, tuesday, wednesday, thursday, friday }

- classrooms = { CLASSROOM1, CCLASSROOM2, CLASSROOM3, CLASSROOM4, CLASSROOM5...CLASSROOMn}

- curricular units = { uC1, CU2, UC3, UC4, CU5...UCn }

- Goal: Assign schedule to lessons;

Constraints:

- There can not be two lessons in the same classroom at the same time;

- There can be at most 3 lessons per day;

- There must be 2 to 4 lessons in a specific classroom;

- Only up to 2 lessons can take place in the morning and up to 2 lessons in the afternoon;

- There must be 1 or 2 lessons online. Online lessons cannot be booked immediately or immediately after a face-to-face lesson. 

- All lessons, with a duration of 2 hours, take place on weekdays.

- A class should not have more than 3 lessons per day.

- Can’t assign a classroom that it is already occupied to another class;

- All lessons must start at least at 8am and end at most by 6pm.

## Characteristics of the task environment:

- This environment is _fully observable_ - An agent’s sensors give it access to the complete state of the environment at each
  point in time. The agent need not maintain any internal state to keep track of the world.

- _Deterministic_ - The next state of the environment is completely determined by the current state and the
  action executed by the agent(s), for eg. when which class we can assign.

- _Sequential_ - It is sequential because when we assign a lesson it affects all the others assignments.

- _Static_ - The environment does not change while an agent is deliberating.

- _Discrete_ - It applies to the state of the environment, to the way time is handled, and to the percepts and actions of
  the agent.


## Performance, Environment, Actuator, Sensors - PEAS

- _Performance_ : Reduce cost of travels, reduce the number of classrooms used.

- _Environment_ : School, classrooms, students, teachers, staff.

- _Actuator_ : Display of schedules, air conditioner

- _Sensors_: None


### New functions created for this CSP:

* atmost_tree_classes_per_day  - returns TRUE if there is no more than 3 lessons in the same day;

* diff_var_two_hours - return True if there are difference of two hours between lessons.

* def build_tuple(dias, horas) - Allow to create a tuple with all combinations of days and hours. eg. build_tuple(('dia1',), (1,2,3,4)), 
Output: ('dia1:1', 'dia1:2', 'dia1:3', 'dia1:4').

* build_string_uc(ucs_h, ucs_s) - Allow to create a tuple with all combinations of days and classrooms. 

* two_to_four_in_same_classroom - returns True when there is only two to for lessons at the same classroom.

* diff_days_lessons - this functions specifies that the lessons of the same UC are in different days return.

* diff_online_lessons -  online lessons cannot be booked immediately or immediately after a face-to-face lesson.

* tot_rooms_online -  there must be 1 or 2 online lessons.





In [ ]:
from csp import *
from notebook import psource, plot_NQueens
%matplotlib inline

# Hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

In [ ]:

def diff_var_two_hours(*values):
    """Returns d"""
    diff = 2
    for ind in range(0, len(values)):
        (indDay, indHour) = values[ind].split(":")
        for indj in range(0, len(values)):
            if(ind == indj): continue
            (indJDay, indJHour) = values[indj].split(":")
            if(indDay == indJDay and abs(float(indHour) - float(indJHour)) < diff):
                return False
    return True

In [ ]:
def build_tuple(dias, horas):
    new_tuple = tuple()
    for dia in dias:
        for hora in horas:
            new_tuple = (*new_tuple, f"{dia}:{hora}")
    return new_tuple

print(build_tuple(('dia1',), (1,2,3,4)))

In [ ]:
def atmost_tree_classes_per_day(*values):
    days_filled = dict()
    for eval_day in values:
        days_filled[eval_day] = days_filled.get(eval_day, 0) + 1
        if days_filled[eval_day] > 3:
            return False
    return list(set(days_filled.values()))[-1] <= 3

In [ ]:
def build_string_uc(ucs_h, ucs_s):

    for t in all_classes:
        ucs_t_h = dict()
        ucs_t_s = dict()
        for uc in all_ucs:
            ucs_t_h[uc] = f"{t}_{uc}_{s}"
            ucs_t_s[uc] = f"{t}_{uc}_{cl}"
        ucs_h[t] = ucs_t_h
        ucs_s[t] = ucs_t_s
    if ucs_h != None and ucs_s != None:
        return True
    else:
        return False


In [ ]:

def build_string_teachers(ucs_p):
    ucs_t_p = dict()
    for p in all_p:
        for uc in all_ucs:
                ucs_t_p[uc] = f"{uc}_{p}"
        ucs_p[p] = ucs_t_p
    if ucs_p != None:
            return True
    else:
            return False

In [ ]:
def two_to_four_in_same_classroom(*values):
    dict_classrooms = dict()
    for classroom in values:
        #if classroom == "ONLINE" : return False
        content = dict_classrooms.get(classroom, 0) + 1
        if content > 4: return False       
        dict_classrooms[classroom] = dict_classrooms.get(classroom, 0) + 1
    return list(set(dict_classrooms.values()))[-1] in range(2, 5) 


print(two_to_four_in_same_classroom("sala1", "sala1", "sala1","sala1", "sala1", "sala2", "sala2", "sala3"))
print(two_to_four_in_same_classroom("sala1", "sala1", "sala2", "sala2", "sala2", "sala1", "sala1", "ONLINE"))

In [ ]:
def diff_days_lessons(value, value_next, online_room):
    day_correspondence = {
        monday : 1,
        tuesday : 2,
        wednesday : 3,
        thursday : 4,
        friday : 5
    }
    if online_tag == online_room: return True
    (indDay, indHour) = value.split(":")
    (indJDay, indJHour) = value_next.split(":")
    return indDay != indJDay# and day_correspondence[indDay] < day_correspondence[indJDay

print(diff_days_lessons("seg:2", "seg:6", "ONLINE"))

In [ ]:
def diff_online_lessons(online_lesson, online_room, lessons):
    """Returns d"""
    diff = 4 if online_room != online_tag else 2
    (indDay, indHour) = online_lesson.split(":")
    (indJDay, indJHour) = lessons.split(":")
    if(indDay == indJDay and abs(float(indHour) - float(indJHour)) < diff):
        return False
    
    return online_room == online_tag

print(diff_online_lessons("segu:2", "SALA","segu:4"))
print(diff_online_lessons("segu:2", "ONLINE","segu:4"))

In [ ]:
def eq_days_online_lessons(value, value_next, online_room):
    day_correspondence = {
        monday : 1,
        tuesady : 2,
        wednesday : 3,
        thursday : 4,
        friday : 5
    }
    if online_room != online_tag: return True
    (indDay, indHour) = value.split(":")
    (indJDay, indJHour) = value_next.split(":")
    return indDay == indJDay

In [25]:
def tot_online_lessons(*values):
    online_rooms = list()
    for room in values:
        if room == online_tag:
            online_rooms.append(room)
    return len(online_rooms) > 0 and len(online_rooms) < 3

In [ ]:
def free_day(*values):
    dict_days = dict()
    for value in values:
        (indDay, indHour) = value.split(':')
        dict_days[indDay] = dict_days.get(indDay, 0) + 1
    return list(set(dict_days.values()))[-1] <= 4

# Assignment of variables


In [26]:
# Days
monday    = 'monday'
tuesday   = 'tuesday'
wednesday = 'wednesday'
thursday  = 'thursday'
friday    =  'friday'

#Period
morning = (8, 9,10,11,12)
evening = (13, 14,15,16,17)

#Class and courses
c1 = 'C1'
c2 = 'C2'

#online tag
online_tag = "ONLINE"

uc1 = 'UC1'
uc2 = 'UC2'
uc3 = 'UC3'
uc4 = 'UC4'
uc5 = 'UC5'

uc1b = 'UC1b'
uc2b = 'UC2b'
uc3b = 'UC3b'
uc4b = 'UC4b'
uc5b = 'UC5b'


#Schedule and classrooms state
s = 'S'
cl = 'C'

#Classrooms
classroom1 = 'CLASSROOM1'
classroom2 = 'CLASSROOM2'
classroom3 = 'CLASSROOM3'
classroom4 = 'CLASSROOM4'
classroom5 = 'CLASSROOM5'

all_classes = [c1, c2]
all_lessons = [l1, l2]
all_uca = [uc1b,uc2b,uc3b,uc4b,uc5b]
all_ucb = [uc1, uc2, uc3, uc4, uc5]
all_ucs = all_uca + all_ucb
all_classrooms = (classroom1, classroom2, classroom3, classroom4, classroom5)
all_days = (monday, tuesday, wednesday, thursday, friday)
ucs_h = {}
ucs_s = {}
ucs_p = {}

build_string_uc(ucs_h, ucs_s)
#build_string_teachers(ucs_p)

True

# Agent Running


In [27]:

domain = {
    ucs_h[c1][uc1]: set(build_tuple((monday, tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc2]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc3]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    ucs_h[c1][uc4]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc5]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    
    ucs_h[c1][uc1b]: set(build_tuple((monday, tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc2b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc3b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    ucs_h[c1][uc4b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c1][uc5b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),

    ucs_h[c2][uc1]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc2]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc3]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    ucs_h[c2][uc4]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc5]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),

    ucs_h[c2][uc1b]: set(build_tuple((monday, tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc2b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc3b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    ucs_h[c2][uc4b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)), 
    ucs_h[c2][uc5b]: set(build_tuple((monday,tuesday, wednesday, thursday, friday), morning + evening)),
    
    ucs_s[c1][uc1] : (classroom1, classroom2, classroom3, classroom4, classroom5, online_tag ),
    ucs_s[c1][uc2] : (classroom1, classroom2, classroom3, classroom4, classroom5, online_tag),
    ucs_s[c1][uc3] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c1][uc4] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c1][uc5] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),

    ucs_s[c1][uc1b] : (classroom1, classroom2, classroom3, classroom4, online_tag),
    ucs_s[c1][uc2b] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c1][uc3b] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c1][uc4b] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c1][uc5b] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),

    ucs_s[c2][uc1] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c2][uc2] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c2][uc3] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c2][uc4] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),
    ucs_s[c2][uc5] : (classroom1, classroom2, classroom3, classroom4, classroom5,online_tag),

    ucs_s[c2][uc1b] :(classroom1, classroom2, classroom3, classroom4, online_tag),
    ucs_s[c2][uc2b] :(classroom1, classroom2, classroom3, classroom4, classroom5, online_tag ),
    ucs_s[c2][uc3b] :(classroom1, classroom2, classroom3, classroom4, classroom5, online_tag ),
    ucs_s[c2][uc4b] :(classroom1, classroom2, classroom3, classroom4, classroom5, online_tag ),
    ucs_s[c2][uc5b] :(classroom1, classroom2, classroom3, classroom4, classroom5, online_tag ),
}

# Ucs da mesma  cadeira separadas que não devem acontencer no mesmo dia
constraints = [
    Constraint((ucs_h[c1][uc1], ucs_h[c1][uc2],ucs_h[c1][uc3], ucs_h[c1][uc4], ucs_h[c1][uc5],), atmost_tree_classes_per_day),
    Constraint((ucs_h[c1][uc1], ucs_h[c1][uc2b], ucs_h[c1][uc3], ucs_h[c1][uc4b], ucs_h[c1][uc5],), free_day),
    Constraint((ucs_h[c1][uc1b], ucs_h[c1][uc2b],ucs_h[c1][uc3b], ucs_h[c1][uc4b], ucs_h[c1][uc5b],), atmost_tree_classes_per_day),
    Constraint((ucs_h[c1][uc1b], ucs_h[c1][uc2],ucs_h[c1][uc3b], ucs_h[c1][uc4], ucs_h[c1][uc5b],), free_day),
    Constraint((ucs_h[c2][uc1], ucs_h[c2][uc2], ucs_h[c2][uc3], ucs_h[c2][uc4], ucs_h[c2][uc5],), atmost_tree_classes_per_day),
    Constraint((ucs_h[c2][uc1b], ucs_h[c2][uc2b], ucs_h[c2][uc3b], ucs_h[c2][uc4b], ucs_h[c2][uc5b],), atmost_tree_classes_per_day),
]

# Listas para constraints de 2 a 4 aulas na mesma sala
classrooms_to_constraints1 = []
classrooms_to_constraints2 = []

for ucc in all_ucs:
    classrooms_to_constraints1.append(ucs_s[c1][ucc])
    classrooms_to_constraints2.append(ucs_s[c2][ucc])


        
 # One or two online classes
constraints.append(Constraint((classrooms_to_constraints1), tot_online_lessons))
constraints.append(Constraint((classrooms_to_constraints2), tot_online_lessons))

# Two to for classes in the same classroom
constraints.append(Constraint((classrooms_to_constraints1),two_to_four_in_same_classroom))
constraints.append(Constraint((classrooms_to_constraints2),two_to_four_in_same_classroom))

for ucc in all_ucs:
    for uccc in all_ucs:
        if ucc == uccc: continue
        constraints.append(Constraint((ucs_h[c1][ucc],  ucs_h[c2][uccc], ucs_s[c1][ucc], ucs_s[c2][uccc]), lambda a, b, c, d: a!=b or c != d))
        constraints.append(Constraint((ucs_h[c1][ucc],  ucs_h[c1][uccc], ucs_s[c1][ucc], ucs_s[c1][uccc]), lambda a, b, c, d: a!=b or c != d))
        constraints.append(Constraint((ucs_h[c2][ucc],  ucs_h[c2][uccc], ucs_s[c2][ucc], ucs_s[c2][uccc]), lambda a, b, c, d: a!=b or c != d))        
        
        constraints.append(Constraint(([ucs_h[c1][ucc], ucs_h[c1][uccc]]), diff_var_two_hours))
        constraints.append(Constraint(([ucs_h[c2][ucc], ucs_h[c2][uccc]]), diff_var_two_hours))
        
#online_lessons
for ucc in all_ucs:
    for uccc in all_ucs:
        if ucc == uccc: continue
        #constraints.append(Constraint((ucs_h[t1][ucc],ucs_h[t1][uccc],ucs_s[t1][uccc], ), eq_days_online_lessons))
        #constraints.append(Constraint((ucs_h[t2][ucc],ucs_h[t2][uccc],ucs_s[t2][uccc], ), eq_days_online_lessons))
       
       

# Lessons that cant be learned at the same day
for classe in all_classes:
    for uca in all_uca:
        for ucb in all_ucb:
            #constraints.append(Constraint((ucs_h[tss][uca], ucs_h[tss][ucb], ucs_s[tss][ucb]), all_diff_constraint))
            constraints.append(Constraint((ucs_h[classe][uca], ucs_h[classe][ucb], ucs_s[classe][ucb]), diff_days_lessons))

# One day without classes - free day
# for classe in all_classes:
#     for uca in all_uca:
#         for ucb in all_ucb:
#             constraints.append(Constraint((ucs_h[classe][uca], ucs_h[classe][ucb],), free_day))

result = NaryCSP(domain, constraints)


# Result
res = ac_solver(result, arc_heuristic=sat_up)


if type(res) != bool:
    dictionary = dict()
    for item in res:
        (t, uc, sh) = item.split("_")
        dictT = dictionary.get(t, dict())
        dictTUC = dictT.get(uc, dict())
        dictTUC[sh] = res[item]
        dictT[uc] = dictTUC
        dictionary[t] =dictT

    import pprint
    from pprint import PrettyPrinter
    pprint.pprint(dictionary)


{'C1': {'UC1': {'C': 'CLASSROOM3', 'S': 'friday:11'},
        'UC1b': {'C': 'CLASSROOM1', 'S': 'thursday:14'},
        'UC2': {'C': 'ONLINE', 'S': 'monday:16'},
        'UC2b': {'C': 'CLASSROOM1', 'S': 'wednesday:15'},
        'UC3': {'C': 'ONLINE', 'S': 'wednesday:11'},
        'UC3b': {'C': 'CLASSROOM3', 'S': 'tuesday:8'},
        'UC4': {'C': 'CLASSROOM1', 'S': 'friday:14'},
        'UC4b': {'C': 'CLASSROOM5', 'S': 'monday:14'},
        'UC5': {'C': 'CLASSROOM1', 'S': 'friday:8'},
        'UC5b': {'C': 'CLASSROOM5', 'S': 'monday:11'}},
 'C2': {'UC1': {'C': 'ONLINE', 'S': 'monday:14'},
        'UC1b': {'C': 'CLASSROOM2', 'S': 'monday:16'},
        'UC2': {'C': 'CLASSROOM1', 'S': 'thursday:15'},
        'UC2b': {'C': 'CLASSROOM3', 'S': 'wednesday:15'},
        'UC3': {'C': 'ONLINE', 'S': 'wednesday:11'},
        'UC3b': {'C': 'CLASSROOM3', 'S': 'friday:8'},
        'UC4': {'C': 'CLASSROOM1', 'S': 'tuesday:16'},
        'UC4b': {'C': 'CLASSROOM3', 'S': 'friday:14'},
        'UC5': {'C'

## Result
![Result](Result.png)

## Conclusion

Due to the observed facts, the conclusion of this work represents for us the achievement of a goal.
We know that Artificial Intelligence can be combined with the various domains of society to solve complex and concrete problems,
with this work we had the opportunity to understand this in a more pragmatic way.
This work was done using the CSP'S algorithm "Constraint Satisfaction Problem", it was a challenge that allowed us to know and overcome our limits and limitations.

### Final considerations

We already tried several different approaches to solve the problem with CSP'S, but we find this algorithm to be the most effective solution for our problem (NaryCSP).
We implemented the functions described below:

* def free_day() - With this function, we pretend to give a free day to the class. Although this function is not being used, due to the time complexity that it carries.

 * def eq_days_online_lessons() - We These function was implemented to force online lessons to happen in the same day.  Although this function is not being used, due to the time complexity that it carries.

